In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
S3_PATH_CONSTITUENCY = {
    'bert': {
        'model': 'v38/constituency/bert-base.pb',
        'dictionary': 'v38/constituency/vocab-bert-base.json',
        'vocab': 'tokenizer/sp10m.cased.bert.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.bert.model',
    },
    'tiny-bert': {
        'model': 'v38/constituency/tiny-bert.pb',
        'dictionary': 'v38/constituency/vocab-tiny-bert.json',
        'vocab': 'tokenizer/sp10m.cased.bert.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.bert.model',
    },
#     'albert': {
#         'model': 'v38/constituency/albert-base.pb',
#         'dictionary': 'v38/constituency/vocab-albert-base.json',
#         'vocab': 'tokenizer/sp10m.cased.v10.vocab',
#         'tokenizer': 'tokenizer/sp10m.cased.v10.model',
#     },
#     'tiny-albert': {
#         'model': 'v38/constituency/albert-tiny.pb',
#         'dictionary': 'v38/constituency/vocab-albert-tiny.json',
#         'vocab': 'tokenizer/sp10m.cased.v10.vocab',
#         'tokenizer': 'tokenizer/sp10m.cased.v10.model',
#     },
#     'xlnet': {
#         'model': 'v40/constituency/xlnet-base.pb',
#         'quantized': 'v40/constituency/xlnet-base.pb.quantized',
#         'dictionary': 'v40/constituency/vocab-xlnet-base.json',
#         'vocab': 'tokenizer/sp10m.cased.v9.vocab',
#         'tokenizer': 'tokenizer/sp10m.cased.v9.model',
#     },
}

In [3]:
for k in S3_PATH_CONSTITUENCY.keys():
    if k not in ['multinomial']:
        print(k)
        os.system(f"wget https://f000.backblazeb2.com/file/malaya-model/{S3_PATH_CONSTITUENCY[k]['model']}")

bert
tiny-bert


In [4]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [5]:
pbs = glob('*.pb')
pbs

['tiny-bert.pb',
 'albert-tiny-similarity.pb',
 'bert-base.pb',
 'xlnet-base-similarity.pb',
 'albert-base-similarity.pb',
 'bert-base-similarity.pb',
 'alxlnet-base-similarity.pb',
 'tiny-bert-similarity.pb']

In [7]:
with tf.gfile.GFile('tiny-bert.pb', "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def)

op = graph.get_operations()
x = []
for i in op:
    try:
        if i.values()[0].shape[-1] == 312:
        #if 'batchnorm/add_1' in i.values()[0].name:
            x.append(i.values())
    except:
        pass
    
x[-100:]

[(<tf.Tensor 'import/bert/encoder/layer_8/attention/output/LayerNorm/batchnorm/mul_2:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/attention/output/LayerNorm/batchnorm/sub:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/attention/output/LayerNorm/batchnorm/add_1:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/intermediate/einsum/Reshape:0' shape=(?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/output/einsum/MatMul:0' shape=(?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/output/einsum/Reshape_1:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/output/add:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/output/add_1:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tensor 'import/bert/encoder/layer_8/output/LayerNorm/moments/SquaredDifference:0' shape=(?, ?, 312) dtype=float32>,),
 (<tf.Tens

In [33]:
mapping = {'albert-base.pb': 'import/bert/encoder/transformer/group_0_11/layer_11/inner_group_0/LayerNorm_1/batchnorm/add_1:0',
          'albert-tiny.pb': 'import/bert/encoder/transformer/group_0_3/layer_3/inner_group_0/LayerNorm_1/batchnorm/add_1:0',
          'bert-base.pb': 'import/bert/encoder/layer_11/output/LayerNorm/batchnorm/add_1:0',
          'tiny-bert.pb': 'import/bert/encoder/layer_11/output/LayerNorm/batchnorm/add_1:0',
          'xlnet-base.pb': 'import/model/transformer/layer_11/ff/LayerNorm/batchnorm/add_1:0'}

In [35]:
mapping[pbs[0]].replace('import/','').replace(':0','')

'bert/encoder/transformer/group_0_11/layer_11/inner_group_0/LayerNorm_1/batchnorm/add_1'

In [37]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    a = [mapping[pb].replace('import/','').replace(':0','')]
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['input_ids', 'word_end_mask'],
                                           ['charts', 'tags'] + a, transforms)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

In [38]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [39]:
quantized = glob('*.pb.quantized')
quantized

['albert-base.pb.quantized',
 'xlnet-base.pb.quantized',
 'albert-tiny.pb.quantized',
 'tiny-bert.pb.quantized',
 'bert-base.pb.quantized']

In [ ]:
# converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()